In [11]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
import torch
from datasets import load_dataset
import os
import pandas as pd
import matplotlib.pyplot as plt
import json

In [12]:
# Cargar el dataset
dataset = load_dataset('json', data_files=r'C:\Users\Lucas\Desktop\Socrates\llm_proyect\data\procesed\dialogos_nlp.json')
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['ID_obra', 'titulo', 'preguntas_socraticas', 'dialogo_socratico', 'preguntas_respuestas_basicas', 'preguntas_implicitas', 'preguntas_respuestas_implicitas'],
        num_rows: 16
    })
})


In [13]:
# Configurar dispositivo como variable global
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("¿CUDA disponible?:", torch.cuda.is_available())
print("Número de GPUs:", torch.cuda.device_count())
print("Nombre de la GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No hay GPU disponible")

¿CUDA disponible?: True
Número de GPUs: 1
Nombre de la GPU: NVIDIA GeForce RTX 3050 Laptop GPU


In [14]:
def check_gpu_memory():
    allocated_memory = torch.cuda.memory_allocated()
    cached_memory = torch.cuda.memory_reserved()
    print(f"Memoria GPU asignada: {allocated_memory / 1024 ** 2:.2f} MB")
    print(f"Memoria GPU reservada: {cached_memory / 1024 ** 2:.2f} MB")
    print(f"Memoria libre   : {torch.cuda.get_device_properties(0).total_memory / 1024 ** 2 - allocated_memory / 1024 ** 2:.2f} MB")

check_gpu_memory()  # Verifica antes de entrenar

Memoria GPU asignada: 477.47 MB
Memoria GPU reservada: 512.00 MB
Memoria libre   : 3618.09 MB


In [15]:
# Cargar modelo y tokenizador
modelo_final_path = r"C:\Users\Lucas\Desktop\Socrates\llm_proyect\src\modelo_obras"
model = AutoModelForCausalLM.from_pretrained(modelo_final_path)
tokenizer = AutoTokenizer.from_pretrained(modelo_final_path)


In [16]:
# Establecer una longitud máxima más manejable
tokenizer.model_max_length = 384
print("Máxima longitud admitida por el modelo:", tokenizer.model_max_length)
model.to(device)

Máxima longitud admitida por el modelo: 384


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50265, 768)
    (wpe): Embedding(512, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50265, bias=False)
)

In [17]:
# Filtrar el dataset para eliminar las entradas vacías
dataset = dataset.filter(lambda example: example['preguntas_respuestas_basicas'] is not None and example['preguntas_respuestas_basicas'] != "")
print(dataset)
print(dataset['train'].column_names)

DatasetDict({
    train: Dataset({
        features: ['ID_obra', 'titulo', 'preguntas_socraticas', 'dialogo_socratico', 'preguntas_respuestas_basicas', 'preguntas_implicitas', 'preguntas_respuestas_implicitas'],
        num_rows: 16
    })
})
['ID_obra', 'titulo', 'preguntas_socraticas', 'dialogo_socratico', 'preguntas_respuestas_basicas', 'preguntas_implicitas', 'preguntas_respuestas_implicitas']


In [18]:
# Nueva función de preprocesamiento para preguntas básicas
def preprocess_preg_resp_function_basicas(examples):
    all_input_ids = []
    all_attention_mask = []
    all_labels = []

    # Procesar cada ejemplo
    for entrada in examples['preguntas_respuestas_basicas']:
        if entrada is None or entrada == "":
            continue
        
        # Procesar cada par pregunta-respuesta como un ejemplo independiente
        for item in entrada:
            pregunta = item.get("pregunta", "")
            respuesta = item.get("respuesta", "")
            
            if not respuesta:
                continue
                
            # Crear el texto completo con formato específico
            texto_completo = f"Pregunta: {pregunta} Respuesta: {respuesta}"
            
            # Tokenizar con truncamiento
            tokenized = tokenizer(
                texto_completo, 
                padding='max_length', 
                truncation=True, 
                max_length=tokenizer.model_max_length,
                return_tensors="pt"
            )
            
            # Guardar los resultados
            all_input_ids.append(tokenized.input_ids[0].tolist())
            all_attention_mask.append(tokenized.attention_mask[0].tolist())
            
            # Para las etiquetas, podemos usar los mismos input_ids
            # (esto depende de tu objetivo de entrenamiento)
            all_labels.append(tokenized.input_ids[0].tolist())
            
            # Opcional: imprimir información para depuración
            print(f"Ejemplo procesado: {texto_completo[:50]}... - Longitud: {len(tokenized.input_ids[0])}")
    
    # Verificar que tenemos datos
    if not all_input_ids:
        print("ADVERTENCIA: No se encontraron pares pregunta-respuesta válidos")
        return {"input_ids": [], "attention_mask": [], "labels": []}
    
    # Crear el diccionario final
    return {
        "input_ids": all_input_ids,
        "attention_mask": all_attention_mask,
        "labels": all_labels
    }

In [19]:
# Nueva función de preprocesamiento para preguntas implícitas
def preprocess_preg_resp_function_implicitas(examples):
    all_input_ids = []
    all_attention_mask = []
    all_labels = []

    # Procesar cada ejemplo
    for entrada in examples['preguntas_respuestas_implicitas']:
        if entrada is None or entrada == "":
            continue
        
        # Procesar cada par pregunta-respuesta como un ejemplo independiente
        for item in entrada:
            pregunta = item.get("pregunta", "")
            respuesta = item.get("respuesta", "")
            
            if not respuesta:
                continue
                
            # Crear el texto completo con formato específico
            texto_completo = f"Pregunta: {pregunta} Respuesta: {respuesta}"
            
            # Tokenizar con truncamiento
            tokenized = tokenizer(
                texto_completo, 
                padding='max_length', 
                truncation=True, 
                max_length=tokenizer.model_max_length,
                return_tensors="pt"
            )
            
            # Guardar los resultados
            all_input_ids.append(tokenized.input_ids[0].tolist())
            all_attention_mask.append(tokenized.attention_mask[0].tolist())
            
            # Para las etiquetas, podemos usar los mismos input_ids
            all_labels.append(tokenized.input_ids[0].tolist())
    
    # Verificar que tenemos datos
    if not all_input_ids:
        print("ADVERTENCIA: No se encontraron pares pregunta-respuesta válidos")
        return {"input_ids": [], "attention_mask": [], "labels": []}
    
    # Crear el diccionario final
    return {
        "input_ids": all_input_ids,
        "attention_mask": all_attention_mask,
        "labels": all_labels
    }

In [20]:
# Aplicar el preprocesamiento
print("Procesando preguntas básicas...")
train_dataset_basicas = dataset['train'].map(
    preprocess_preg_resp_function_basicas, 
    batched=True,
    remove_columns=dataset['train'].column_names  # Eliminar columnas originales
)

Procesando preguntas básicas...


Map:   0%|          | 0/16 [00:00<?, ? examples/s]

Ejemplo procesado: Pregunta: Sócrates Alcibíades, ¿vas a orar en este... - Longitud: 384
Ejemplo procesado: Pregunta: Sócrates A mí me parece que hay materia ... - Longitud: 384
Ejemplo procesado: Pregunta: Sócrates Y bien, ¿no te parece que la or... - Longitud: 384
Ejemplo procesado: Pregunta: Sócrates ¿Pero el delirio te parece lo c... - Longitud: 384
Ejemplo procesado: Pregunta: Sócrates ¿No te parece que los hombres s... - Longitud: 384
Ejemplo procesado: Pregunta: Sócrates Además, ¿no hay hombres sanos? ... - Longitud: 384
Ejemplo procesado: Pregunta: Sócrates ¿No son los mismos? Respuesta: ... - Longitud: 384
Ejemplo procesado: Pregunta: Sócrates ¿Hay otros que no están ni en u... - Longitud: 384
Ejemplo procesado: Pregunta: Sócrates Pero qué, ¿piensas lo mismo res... - Longitud: 384
Ejemplo procesado: Pregunta: Sócrates ¿Te pregunto si te parece impre... - Longitud: 384
Ejemplo procesado: Pregunta: Sócrates ¿Luego es indispensable ser lo ... - Longitud: 384
Ejemplo procesado: Pr

Map: 100%|██████████| 16/16 [00:00<00:00, 37.01 examples/s]


In [21]:
print("Procesando preguntas implícitas...")
train_dataset_implicitas = dataset['train'].map(
    preprocess_preg_resp_function_implicitas, 
    batched=True,
    remove_columns=dataset['train'].column_names  # Eliminar columnas originales
)

Procesando preguntas implícitas...


Map: 100%|██████████| 16/16 [00:00<00:00, 22.06 examples/s]


In [22]:
# Verificar la estructura de los datasets procesados
print("Dataset de preguntas básicas:")
print(train_dataset_basicas)
print(train_dataset_basicas.column_names)
if len(train_dataset_basicas) > 0:
    print("Ejemplo:", train_dataset_basicas[0])

print("Dataset de preguntas implícitas:")
print(train_dataset_implicitas)
print(train_dataset_implicitas.column_names)
if len(train_dataset_implicitas) > 0:
    print("Ejemplo:", train_dataset_implicitas[0])

Dataset de preguntas básicas:
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 770
})
['input_ids', 'attention_mask', 'labels']
Ejemplo: {'input_ids': [2, 25820, 80, 415, 45386, 771, 6457, 351, 12505, 66, 1486, 5531, 320, 651, 322, 334, 533, 11023, 85, 3440, 1596, 80, 771, 6457, 351, 12505, 12176, 66, 415, 45386, 68, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264

In [23]:
# Convertir las columnas a formato PyTorch
train_dataset_basicas.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
train_dataset_implicitas.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

In [24]:
print(train_dataset_basicas[4])  # Verifica las entradas después de tokenizar
print(train_dataset_implicitas[8])  # Verifica las entradas después de tokenizar

{'input_ids': tensor([    2, 25820,    80,   415, 45386,  1486,   836,   510,  1919,   341,
          364,  3149,   644,  1279, 13691,   408,   342,  1071, 48691,   408,
           85,  3440,  1596,    80,   771,  6457,   351, 12505,  3356,   445,
          638,    68, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264,
        50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264,
        50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264,
        50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264,
        50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264,
        50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264,
        50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264,
        50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264,
        50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264, 50264,
        50264, 50264, 50264, 50264, 50264, 50264, 

In [25]:
print(type(train_dataset_implicitas))
print(train_dataset_implicitas.column_names)
print(train_dataset_implicitas.features)
print(train_dataset_implicitas[1]) 

<class 'datasets.arrow_dataset.Dataset'>
['input_ids', 'attention_mask', 'labels']
{'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}
{'input_ids': tensor([    2, 25820,    80, 17603,  7792,   342, 12792,  3217,    66,  1278,
          313,  1910,   387,  8582,    66,   332,  2498,   342,   332,   393,
        45296, 16757,    66,   442,   895,   689,  2349,   341,  8975,   332,
         3425,   467,  8313,    66, 27525,   341,   403, 16430,  1111, 45220,
          383,   442,   341,   981,   389,  1582,   341, 35679,   313, 14777,
        37996,    66,  1486, 10878,   403,  1649, 23926,   592,  5702,   313,
          514,  3425,    66,   342,   403,  3226,  5702, 13396,   341,  8297,
        14319,   514, 19440,   342,   514,   530, 21269,    85,  3440,  1596,
           80,   400,  3775,   

In [26]:
print(type(train_dataset_basicas))
print(train_dataset_basicas.column_names)
print(train_dataset_basicas.features)
print(train_dataset_basicas[1]) 

<class 'datasets.arrow_dataset.Dataset'>
['input_ids', 'attention_mask', 'labels']
{'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}
{'input_ids': tensor([    2, 25820,    80,   415, 45386,   400,  3011,   503,  1919,   341,
          733,  3382,   394,  4140, 24716,    66,   921,    66,  2894,   318,
         1969,   313, 33338, 11674,  1486,   552, 14068,   341,   619,   387,
         1692,   341, 15012,   320,   364, 20427,    66,  1351,   334,  1757,
           66,  1351,   334,  7922,    66,   733,  1763,   341,   361,   683,
        34453,   342,  1312,   341,   361,   683, 22494,    66,   342,   341,
          719,  5072, 27200,   466, 25328,   328,  2578, 12672, 35676,    85,
         3440,  1596,    80,   771,  6457,   351, 12505, 12176,   442,  2768,
           68, 50264, 50264, 50

In [27]:
from transformers import GPT2LMHeadModel, TrainingArguments, Trainer

# Argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./output",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    save_steps=500,
    save_total_limit=2,
    logging_steps=100,
    evaluation_strategy="no",
    fp16=True,
)




c:\Users\Lucas\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [28]:
# Entrenador para preguntas_respuestas_basicas
trainer_basicas = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_basicas,
    tokenizer=tokenizer,
)



C:\Users\Lucas\AppData\Local\Temp\ipykernel_11736\2806936238.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_basicas = Trainer(


In [29]:
# Entrenador para preguntas_respuestas_implicitas
trainer_implicitas = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_implicitas,
    tokenizer=tokenizer,
)

C:\Users\Lucas\AppData\Local\Temp\ipykernel_11736\4278263293.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_implicitas = Trainer(


In [30]:
# Iniciar el entrenamiento para ambos datasets
trainer_basicas.train()  
trainer_implicitas.train()  

Step,Training Loss
100,9.606800
200,3.227000
300,0.367400
400,0.333200
500,0.222200
600,0.226600
700,0.234600
800,0.202300
900,0.134300
1000,0.134800


Step,Training Loss
100,0.303600
200,0.301300
300,0.289700
400,0.302200
500,0.308700
600,0.326800
700,0.294000
800,0.280300
900,0.277300
1000,0.218500


TrainOutput(global_step=2616, training_loss=0.21176382999536825, metrics={'train_runtime': 4495.1474, 'train_samples_per_second': 1.163, 'train_steps_per_second': 0.582, 'total_flos': 1024722026496000.0, 'train_loss': 0.21176382999536825, 'epoch': 3.0})

In [32]:
import os

# Definir rutas
modelo_final_path = r"C:\Users\Lucas\Desktop\Socrates\llm_proyect\src\modelo_dialogos"
checkpoints_path = r"C:\Users\Lucas\Desktop\Socrates\llm_proyect\data\checkpoints"

# Asegurar que las carpetas existen
os.makedirs(modelo_final_path, exist_ok=True)
os.makedirs(checkpoints_path, exist_ok=True)

In [33]:
# Guardar el modelo
model.save_pretrained(modelo_final_path)

# Guardar el tokenizador
tokenizer.save_pretrained(modelo_final_path)

('C:\\Users\\Lucas\\Desktop\\Socrates\\llm_proyect\\src\\modelo_dialogos\\tokenizer_config.json',
 'C:\\Users\\Lucas\\Desktop\\Socrates\\llm_proyect\\src\\modelo_dialogos\\special_tokens_map.json',
 'C:\\Users\\Lucas\\Desktop\\Socrates\\llm_proyect\\src\\modelo_dialogos\\vocab.json',
 'C:\\Users\\Lucas\\Desktop\\Socrates\\llm_proyect\\src\\modelo_dialogos\\merges.txt',
 'C:\\Users\\Lucas\\Desktop\\Socrates\\llm_proyect\\src\\modelo_dialogos\\added_tokens.json',
 'C:\\Users\\Lucas\\Desktop\\Socrates\\llm_proyect\\src\\modelo_dialogos\\tokenizer.json')

In [31]:
input_text = "¿Qué es la justicia?"
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

output = model.generate(**inputs, max_length=100)
print(tokenizer.decode(output[0], skip_special_tokens=True))

¿Qué es la justicia? Respuesta: La justicia es una cosa y la injusticia otra. Sócrates Y cuando un hombre se pone en la palestra, ¿no es al mismo tiempo y de una manera u otra a la vez, como un hombre se pone en la palestra? Sócrates Y cuando un hombre se pone en la palestra, ¿no es al mismo tiempo y de una manera u otra a la vez, como un
